# Gym Ignition Example

## Installation

In [0]:
# Disable stdout (use output.show() to print it)
%%capture --no-stderr output

# UPDATE CMAKE
!\
    wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | sudo apt-key add - &&\
    apt-add-repository 'deb https://apt.kitware.com/ubuntu/ bionic main' &&\
    apt-get -qq update &&\
    apt-get -qq install cmake

# INSTALL SWIG
!\
    apt-get -qq install -y --no-install-recommends \
        autotools-dev \
        automake \
        bison \
        libpcre3-dev &&\
    cd /tmp/ &&\
    rm -rf swig &&\
    git clone --depth 1 -b rel-4.0.0 https://github.com/swig/swig.git &&\
    cd swig &&\
    sh autogen.sh &&\
    ./configure &&\
    make -j8 &&\
    make install

# INSTALL IGNITION LIBRARIES
!\
    apt-get -qq update &&\
    echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list &&\
    echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-prerelease `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-prerelease.list &&\
    echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-nightly `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-prerelease.list &&\
    wget http://packages.osrfoundation.org/gazebo.key -O - | apt-key add - &&\
    apt-get -qq update &&\
    apt-get -qq install -y --no-install-recommends \
        libgflags-dev \
        libignition-cmake2-dev \
        libignition-plugin-dev \
        libignition-math6-dev \
        libignition-common3-dev \
        libignition-transport7-dev \
        libignition-msgs4-dev \
        libignition-tools-dev \
        libignition-fuel-tools3-dev \
        libsdformat8-dev \
        libignition-physics-dev \
        libignition-rendering-dev \
        libignition-sensors2-dev \
        libignition-gui2-dev

# INSTALL IGNITION GAZEBO
!cd /tmp &&\
    rm -rf ign-gazebo &&\
    git clone --depth 1 https://github.com/diegoferigo/ign-gazebo &&\
    cd ign-gazebo && mkdir -p build && cd build &&\
    cmake -DCMAKE_BUILD_TYPE="Release" -DBUILD_TESTING:BOOL=OFF .. &&\
    cmake --build . --target install -- -j8 &&\
    rm -r /tmp/ign-gazebo

# INSTALL GYM IGNITION
!export PATH=/usr/bin:$PATH &&\
    cd /tmp &&\
    rm -rf gym-ignition &&\
    git clone https://github.com/robotology/gym-ignition &&\
    cd gym-ignition && mkdir -p build && cd build &&\
    cmake .. &&\
    cmake --build . --target install -- -j8 &&\
    cd .. &&\
    pip3 install .

In [0]:
# CONFIGURE PYTHON PATH
import sys
sys.path.append('/usr/local/lib/gympp/bindings')

# CONFIGURE ENV
import os
os.environ['IGN_GAZEBO_SYSTEM_PLUGIN_PATH'] = "/usr/local/lib/gympp/plugins"
os.environ['IGN_GAZEBO_RESOURCE_PATH'] = "/usr/local/share/gympp/gazebo/worlds:/usr/local/share/gympp/gazebo/models"

In [3]:
%%capture --no-stderr --no-stdout output

import gym
import gym_ignition
import gym_ignition.utils.logger as logger

# Set gym verbosity
logger.set_level(gym.logger.INFO)

logger.debug("Initializing the environment")

# Create the environment
# env = gym.make("CartPole-v1")
# env = gym.make("CartPoleIgnition-v0")
env = gym.make("CartPoleIgnitionPython-v0")

# Initialize the seed
env.seed(42)

for epoch in range(10):
    # Reset the environment
    observation = env.reset()

    # Initialize returned values
    done = False
    totalReward = 0

    while not done:
        # Execute a random action
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)

        # Render the environment
        # env.render('human')

        # Accumulate the reward
        totalReward += reward

        # Print the observation
        msg = ""
        for value in observation:
            msg += "\t%.6f" % value
        logger.debug(msg)

    logger.info("Total reward for episode #{}: {}".format(epoch, totalReward))

env.close()


INFO: Making new env: CartPoleIgnitionPython-v0
INFO: Total reward for episode #0: 39.0
INFO: Total reward for episode #1: 27.0
INFO: Total reward for episode #2: 32.0
INFO: Total reward for episode #3: 32.0
INFO: Total reward for episode #4: 28.0
INFO: Total reward for episode #5: 30.0
INFO: Total reward for episode #6: 25.0
INFO: Total reward for episode #7: 53.0
INFO: Total reward for episode #8: 20.0
INFO: Total reward for episode #9: 45.0
